In [1]:
#Library Import

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import altair
import scipy.stats as stats
from scipy.stats import norm

# import seaborn as sns
# from sklearn.neighbors import KernelDensity
# from bokeh.models import ColumnDataSource, Label, PrintfTickFormatter
# from bokeh.palettes import Dark2_5 as colors
# from bokeh.plotting import figure, show
# from bokeh.sampledata.cows import data as df
# from bokeh.io import output_notebook


In [2]:
#copied from main.ipynb
# #import pandas as pd


df_course = pd.read_csv('assets/student.course.csv')
df_record = pd.read_csv('assets/student.record.csv')
df_term = pd.read_table('assets/term.table.txt', delimiter="\t").fillna('Unknown')



df_term.columns = ['TERM_ID', 'TERM_DESCRIPTION']

terms = ['Fall', 'Winter', 'Summer', 'Spring', 'Unknown']
df_term['TERM_NAME'] = df_term['TERM_DESCRIPTION'].apply(lambda x: list(filter(lambda y: y!='', [t if t in x else '' for t in terms]))[0])
df_term['TERM_YEAR'] = df_term['TERM_DESCRIPTION'].apply(lambda x: x[-4:] if x[-4:].isnumeric() else None)

merge_course_df = pd.merge(
                    df_course,
                    df_term, 
                    how='left',
                    left_on='TERM',
                    right_on='TERM_ID')

terms_student = ['ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM']
merge_record_df = df_record.copy()
for term in terms_student:
    
    merge_record_df = pd.merge(
                        merge_record_df,
                        df_term, 
                        how='left',
                        left_on=term,
                        right_on='TERM_ID')
    
    

    merge_record_df.rename(columns={'TERM_ID': term + '_' + 'TERM_ID', 
                            'TERM_DESCRIPTION': term + '_' + 'TERM_DESCRIPTION', 
                            'TERM_NAME': term + '_' + 'TERM_NAME', 
                            'TERM_YEAR': term + '_' + 'TERM_YEAR'}, inplace=True)    
    
 


# df_term
# df_course
# df_record
# merge_course_df
merge_record_df

    
course_columns = ['ANONID', 'SUBJECT', 'CATALOG_NBR', 'GRD_PTS_PER_UNIT', 'GPAO', 'DIV',
                  'ANON_INSTR_ID', 'TERM', 'TERM_ID', 'TERM_DESCRIPTION', 'TERM_NAME', 'TERM_YEAR']

record_columns = ['MAJOR3_DESCR', 'MAJOR2_DESCR', 'MAJOR1_DESCR', 'HSGPA', 'LAST_ACT_ENGL_SCORE', 'LAST_ACT_MATH_SCORE',
                  'LAST_ACT_READ_SCORE', 'LAST_ACT_SCIRE_SCORE', 'LAST_ACT_COMP_SCORE', 'LAST_SATI_VERB_SCORE', 
                  'LAST_SATI_MATH_SCORE', 'LAST_SATI_TOTAL_SCORE', 'SEX', 'STDNT_GROUP1', 'STDNT_GROUP2', 'MAJOR1_DEPT', 
                  'MAJOR2_DEPT', 'MAJOR3_DEPT', 'ANONID', 'ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM', 
                  'MAJOR3_TERM_TERM_ID', 'MAJOR3_TERM_TERM_DESCRIPTION', 'MAJOR3_TERM_TERM_NAME', 'MAJOR3_TERM_TERM_YEAR']   

# merge_course_df.describe()
# merge_record_df.describe()


# https://www.bls.gov/oes/tables.htm
# https://data.census.gov/table/ACSST1Y2015.S0101
# https://data.census.gov/table/ACSST1Y2015.S1501?t=Education&g=040XX00US06
# merge_record_df.sort_values(by=['MAJOR3_TERM_TERM_YEAR'], ascending=[True])['MAJOR3_TERM_TERM_YEAR'].unique()
# 1992 - 2015


# Occupational data by gender
# https://www.bls.gov/cps/cps_aa2015.htm

# https://www.axios.com/2020/06/30/half-us-population-unemployed
# the labor market is only at 50%?


merge_record_df[['ADMIT_TERM','ADMIT_TERM_TERM_ID','ADMIT_TERM_TERM_DESCRIPTION','ADMIT_TERM_TERM_NAME']]


,ADMIT_TERM,ADMIT_TERM_TERM_ID,ADMIT_TERM_TERM_DESCRIPTION,ADMIT_TERM_TERM_NAME
0,NaN,NaN,NaN,NaN
1,110.0,110.0,Fall 2008,Fall
2,63.0,63.0,Fall 1998,Fall
3,106.0,106.0,Fall 2007,Fall
4,83.0,83.0,Fall 2002,Fall
...,...,...,...,...
138883,NaN,NaN,NaN,NaN
138884,126.0,126.0,Fall 2012,Fall
138885,98.0,98.0,Fall 2005,Fall
138886,NaN,NaN,NaN,NaN


In [3]:
merge_record_df.columns

Index(['MAJOR3_DESCR', 'MAJOR2_DESCR', 'MAJOR1_DESCR', 'HSGPA',
       'LAST_ACT_ENGL_SCORE', 'LAST_ACT_MATH_SCORE', 'LAST_ACT_READ_SCORE',
       'LAST_ACT_SCIRE_SCORE', 'LAST_ACT_COMP_SCORE', 'LAST_SATI_VERB_SCORE',
       'LAST_SATI_MATH_SCORE', 'LAST_SATI_TOTAL_SCORE', 'SEX', 'STDNT_GROUP1',
       'STDNT_GROUP2', 'MAJOR1_DEPT', 'MAJOR2_DEPT', 'MAJOR3_DEPT', 'ANONID',
       'ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM',
       'ADMIT_TERM_TERM_ID', 'ADMIT_TERM_TERM_DESCRIPTION',
       'ADMIT_TERM_TERM_NAME', 'ADMIT_TERM_TERM_YEAR', 'MAJOR1_TERM_TERM_ID',
       'MAJOR1_TERM_TERM_DESCRIPTION', 'MAJOR1_TERM_TERM_NAME',
       'MAJOR1_TERM_TERM_YEAR', 'MAJOR2_TERM_TERM_ID',
       'MAJOR2_TERM_TERM_DESCRIPTION', 'MAJOR2_TERM_TERM_NAME',
       'MAJOR2_TERM_TERM_YEAR', 'MAJOR3_TERM_TERM_ID',
       'MAJOR3_TERM_TERM_DESCRIPTION', 'MAJOR3_TERM_TERM_NAME',
       'MAJOR3_TERM_TERM_YEAR'],
      dtype='object')

In [5]:
columns = ['LAST_ACT_ENGL_SCORE', 'LAST_ACT_MATH_SCORE', 'LAST_ACT_READ_SCORE',
       'LAST_ACT_SCIRE_SCORE', 'LAST_ACT_COMP_SCORE', 'LAST_SATI_VERB_SCORE',
       'LAST_SATI_MATH_SCORE', 'LAST_SATI_TOTAL_SCORE', 'SEX', 'STDNT_GROUP1',
       'STDNT_GROUP2','ADMIT_TERM_TERM_YEAR']

In [6]:
merge_record_df[columns].head()

,LAST_ACT_ENGL_SCORE,LAST_ACT_MATH_SCORE,LAST_ACT_READ_SCORE,LAST_ACT_SCIRE_SCORE,LAST_ACT_COMP_SCORE,LAST_SATI_VERB_SCORE,LAST_SATI_MATH_SCORE,LAST_SATI_TOTAL_SCORE,SEX,STDNT_GROUP1,STDNT_GROUP2,ADMIT_TERM_TERM_YEAR
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN
1,25.0,22.0,29.0,28.0,26.0,NaN,NaN,NaN,F,NaN,NaN,2008
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,E,NaN,1998
3,33.0,32.0,28.0,30.0,31.0,NaN,NaN,NaN,F,NaN,NaN,2007
4,24.0,22.0,18.0,18.0,21.0,610.0,540.0,1150.0,F,C,NaN,2002
